In [23]:
import pandas as pd
import numpy as np

from bokeh.io import output_notebook
from bokeh.plotting import figure, show, output_file
from bokeh.layouts import column

output_notebook()

Loading BokehJS ...

In [2]:
colocations = pd.read_csv("/data/Linde_Intel_AI_Challenge_Nov2018/colocations.csv")
customer_details = pd.read_csv("/data/Linde_Intel_AI_Challenge_Nov2018/customer_details.csv")
deliveries = pd.read_csv("/data/Linde_Intel_AI_Challenge_Nov2018/deliveries.csv")
level_readings = pd.read_csv("/data/Linde_Intel_AI_Challenge_Nov2018/level_readings.csv")

## Convert to date and time

In [6]:
deliveries["del_date"] = pd.to_datetime(deliveries["DELIVERY_DATE"]).dt.date
deliveries["del_time"] = pd.to_datetime(deliveries["DELIVERY_DATE"]).dt.time

level_readings["read_date"] = pd.to_datetime(level_readings["ON_DATE_TIME"]).dt.date
level_readings["read_time"] = pd.to_datetime(level_readings["ON_DATE_TIME"]).dt.time

# Co locations

| ... | ... | ... |
| --- | --- | --- |
|VESSEL_ID 1 |Identifier of primary storage vessel |VARCHAR|
|VESSEL_ID 2 - 7 |Identifier of any (up to seven) co-located storage vessels |VARCHAR|

In [7]:
colocations.head()

,VESSEL_ID_1,VESSEL_ID_2,VESSEL_ID_3,VESSEL_ID_4,VESSEL_ID_5,VESSEL_ID_6,VESSEL_ID_7
0,BR-218373,NaN,NaN,NaN,NaN,NaN,NaN
1,BR-216975,NaN,NaN,NaN,NaN,NaN,NaN
2,BR-216968,NaN,NaN,NaN,NaN,NaN,NaN
3,BR-217297,NaN,NaN,NaN,NaN,NaN,NaN
4,BR-217930,NaN,NaN,NaN,NaN,NaN,NaN


# Customer deets

|INST_ID                     | Installation Identifier                                         | INT      |
| --- | --- | --- |
|CNTRY_UN_COUNTRY_CODE b     | Two-digit standard country identifier                           | CHAR (2) |
|PROVINCE                    |  Geographic location Province/State/County                      | VARCHAR  |
|PRD_ID                      | Abbreviation of Product Name                                    | CHAR (3) |
|MAXIMUM_PRODUCT_CAPACITY    | Maximum capacity of the storage vessel                          | INT (KG) |
|VESSEL_ID                   | Concatenation of Country identifier and Installation identifier | VARCHAR  |
|MKT_SEGMENT                 | Market Segment Classification                                   | VARCHAR  |
|ISIC_CODE (where available) |  International Standard Industrial Classification (ISIC) code   | INT      |
|ISIC_DESC                   | ISIC Code Description                                           | VARCHAR  |

In [8]:
customer_details.head()

,INST_ID,CNTRY_UN_COUNTRY_CODE,PROVINCE,PRD_ID,MAXIMUM_PRODUCT_CAPACITY,VESSEL_ID,MKT_SEGMENT,ISIC_CODE,ISIC_DESCRIPTION
0,216705,BR,PB,LOX,2195,BR-216705,Healthcare,8610.0,Hospital activities
1,216706,BR,SP,LOX,13075,BR-216706,Chemistry & Energy,3822.0,Treatment and disposal of hazardous wast
2,216721,BR,SP,LOX,3567,BR-216721,Manufacturing Industries,2910.0,Manufacture of motor vehicles
3,216729,BR,SP,LAR,13852,BR-216729,Metals & Glass,2610.0,Manufacture of electronic components and
4,216732,BR,PE,LOX,2195,BR-216732,Healthcare,8610.0,Hospital activities


In [9]:
len(customer_details["VESSEL_ID"].unique())

2347

# Deliveries

|...|...|...|
|--- | ---| ---|
|INST_ID |Installation Identifier |INT
|DELIVERY_DATE |Date/time of delivery |YYYY-MM-DD HH:MM
|DELIVERED_VOLUME | Amount of product delivered |INT (KG)

In [10]:
deliveries.head()

,DELIVERY_DATE,DELIVERED_VOLUME,VESSEL_ID,del_date,del_time
0,3/13/2016 1:34,12362,UK-3749,2016-03-13,01:34:00
1,3/13/2016 2:18,11564,UK-3806,2016-03-13,02:18:00
2,3/13/2016 6:48,3103,UK-41488,2016-03-13,06:48:00
3,3/13/2016 7:07,3683,UK-63168,2016-03-13,07:07:00
4,3/13/2016 7:24,6545,UK-2435,2016-03-13,07:24:00


# Level headings

|...|...|...|
|--- | ---| ---|
|INST_ID |Installation Identifier| INT
|ON_DATE_TIME| Date/time of reading| YYYY-MM-DD HH:MM
|INST_PRODUCT_AMOUNT |Instantaneous product level reading | INT (KG)

In [11]:
level_readings.head()

,COUNTRY_CODE,INST_ID,INST_PRODUCT_AMOUNT,ON_DATE_TIME,VESSEL_ID,read_date,read_time
0,MY,62,13870,2017-10-02T00:59:00Z,MY-62,2017-10-02,00:59:00
1,MY,62,18330,2017-10-02T01:59:00Z,MY-62,2017-10-02,01:59:00
2,MY,62,15367,2017-10-02T02:59:00Z,MY-62,2017-10-02,02:59:00
3,MY,62,13805,2017-10-02T03:59:00Z,MY-62,2017-10-02,03:59:00
4,MY,62,27870,2017-10-02T04:59:00Z,MY-62,2017-10-02,04:59:00


# Delivery and level reading based on vessel id

In [33]:
vessel_id = "UK-3749"
vessel_id_del_data = deliveries[deliveries["VESSEL_ID"] == vessel_id].sort_values(by=["del_date"])
vessel_id_level_data = level_readings[level_readings["VESSEL_ID"] == vessel_id].sort_values(by=["read_date"])

col_plot = column()

p = figure(plot_width=800, plot_height=600, title="Deliveries for Vessel ID: "+vessel_id, x_axis_type='datetime')

p.line(vessel_id_del_data["del_date"], vessel_id_del_data["DELIVERED_VOLUME"], color='#A6CEE3', line_width=3)
p.square(vessel_id_del_data["del_date"], vessel_id_del_data["DELIVERED_VOLUME"], color='#A6CEE3', line_width=3)

col_plot.children.append(p)

p = figure(plot_width=970, plot_height=600, title="Level readings for Vessel ID: "+vessel_id, x_axis_type='datetime')

p.line(vessel_id_level_data["read_date"], vessel_id_level_data["INST_PRODUCT_AMOUNT"], color='green', line_width=3)
p.square(vessel_id_level_data["read_date"], vessel_id_level_data["INST_PRODUCT_AMOUNT"], color='green', line_width=3)

col_plot.children.append(p)

show(col_plot)

In [35]:
vessel_id_level_data.shape

(16758, 7)

## To check if all data points have unique timestamps

In [43]:
count = 0
for vessel_id in vessel_id_level_data["VESSEL_ID"].unique():
    # vessel_id_del_data = deliveries[deliveries["VESSEL_ID"] == vessel_id].sort_values(by=["del_date"])
    vessel_id_level_data = level_readings[level_readings["VESSEL_ID"] == vessel_id].sort_values(by=["read_date"])
    if len(vessel_id_level_data["ON_DATE_TIME"].unique()) == vessel_id_level_data.shape[0]:
        count = count + 1
        
if count == len(vessel_id_level_data["VESSEL_ID"].unique()):
    print("ALL UNIQUE")
else:
    print(NOT)

ALL UNIQUE


In [42]:
vessel_id_level_data[vessel_id_level_data["read_date"] == vessel_id_level_data["read_date"].unique()[0]].sort_values(by="read_time")

,COUNTRY_CODE,INST_ID,INST_PRODUCT_AMOUNT,ON_DATE_TIME,VESSEL_ID,read_date,read_time
4101190,UK,3749,23807,2016-10-13T01:00:00Z,UK-3749,2016-10-13,01:00:00
4101191,UK,3749,23773,2016-10-13T02:00:00Z,UK-3749,2016-10-13,02:00:00
4101192,UK,3749,23749,2016-10-13T03:00:00Z,UK-3749,2016-10-13,03:00:00
4101193,UK,3749,23715,2016-10-13T04:00:00Z,UK-3749,2016-10-13,04:00:00
4101194,UK,3749,23676,2016-10-13T05:00:00Z,UK-3749,2016-10-13,05:00:00
4101195,UK,3749,23639,2016-10-13T06:00:00Z,UK-3749,2016-10-13,06:00:00
4101196,UK,3749,23642,2016-10-13T07:00:00Z,UK-3749,2016-10-13,07:00:00
4101197,UK,3749,23573,2016-10-13T08:00:00Z,UK-3749,2016-10-13,08:00:00
4101198,UK,3749,23557,2016-10-13T09:00:00Z,UK-3749,2016-10-13,09:00:00
4101199,UK,3749,23518,2016-10-13T10:00:00Z,UK-3749,2016-10-13,10:00:00
